In [ ]:
#Importing the required packages 

from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import time 
import pandas as pd


#Function for web scraping 



def scraper_2(driver):

    gifts = driver.find_elements(By.CSS_SELECTOR, 'figure[data-test="product"]')

    #print(gifts) 
    #print(len(gifts))
    
    gift_ideas = []
    gift_prices = []
    gift_links = []
    gift_images = []

    for gift in gifts:
            #Iterate over every item of information
        gift_name = gift.find_element(By.CSS_SELECTOR, "h2[aria-label]").text
        
        if gift_name not in total_ideas:
            gift_ideas.append(gift_name)
            print(gift_name)
                #Selects all the gift names 
                #Converts the HTML into text and saves it 

            try:
                gift_prices_pound = gift.find_element(By.CLASS_NAME, "a-price-whole").text
                gift_prices_pence = gift.find_element(By.CLASS_NAME, "a-price-fraction").text

                price = gift_prices_pound + "." + gift_prices_pence
                gift_prices.append(price)
                print(price)
                    #Finds the price normally

            except:
                price = gift.find_element(By.CLASS_NAME, "a-color-base")
                gift_prices.append(price)
                print(price)
                    #This finds the price if not in the normal location 

            gift_link = gift.find_element(By.TAG_NAME, "a").get_attribute("href")
            gift_links.append(gift_link)
            print(gift_link)
                #Seletcs the link for the products 
                
            gift_image = gift.find_element(By.CLASS_NAME, "s-image").get_attribute("src")
            gift_images.append(gift_image)
            print(gift_image)

            print()
            
        else:
            continue
        
    return gift_ideas, gift_prices, gift_links, gift_images




#Applying this Function

total_ideas = []
total_prices = []
total_links = []
total_images = []

driver = webdriver.Chrome()

#url = "https://www.amazon.co.uk/gcx/Gifts-for-everyone/gfhz/?scrollState=eyJpdGVtSW5kZXgiOjAsInNjcm9sbE9mZnNldCI6Njg4LjM2MjQ4Nzc5Mjk2ODh9&sectionManagerState=eyJzZWN0aW9uVHlwZUVuZEluZGV4Ijp7ImFtYWJvdCI6MH19"
#url = "https://www.amazon.co.uk/gcx/Gifts-for-Men/gfhz/category?categoryId=adult-male&scrollState=eyJpdGVtSW5kZXgiOjAsInNjcm9sbE9mZnNldCI6MzkwLjgwMDAxODMxMDU0Njl9&sectionManagerState=eyJzZWN0aW9uVHlwZUVuZEluZGV4Ijp7ImFtYWJvdCI6MH19"
#url = "https://www.amazon.co.uk/gcx/Gifts-for-Women/gfhz/category?categoryId=adult-female&scrollState=eyJpdGVtSW5kZXgiOjAsInNjcm9sbE9mZnNldCI6MzkwLjgwMDAxODMxMDU0Njl9&sectionManagerState=eyJzZWN0aW9uVHlwZUVuZEluZGV4Ijp7ImFtYWJvdCI6MH19"
    #Each of these links is where I got the Amazon Data from 
    #Repeat below for each of the links 

driver.get(url)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #Scrolls the webpage to the bottom
    time.sleep(2)
    
    ideas, prices, links, images = scraper_2(driver)
    
    total_ideas.extend(ideas)
    total_links.extend(links)
    total_prices.extend(prices)
    total_images.extend(images)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("Page Scraping Complete")
        print()
        
        time.sleep(1)
        
        #ideas, prices, links = scraper_test(driver)
            #Need to remove to prevent overlap
        
        break
        
    last_height = new_height
    
driver.quit()


#Saving the data 

gift_data = {
    "Name" : total_ideas,
    "Price" : total_prices,
    "Link" : total_links,
    "Image URL" : total_images}

pd.set_option('display.max_colwidth', None)
    #This prevents the automatic truncation in pandas dataframes 

gift_dataframe = pd.DataFrame(gift_data)

gift_dataframe.to_csv("Gift_Data.csv", index=False, encoding="utf-8")